In [38]:
#We are analyzing gaming channels, basically 'Hello World'

import os
import google.oauth2.credentials

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

In [39]:
# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
#client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [40]:
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def channels_list_by_username(service, **kwargs):
  results = service.channels().list(
    **kwargs
  ).execute()
  
  print('This channel\'s ID is %s. Its title is %s, and it has %s views.' %
       (results['items'][0]['id'],
        results['items'][0]['snippet']['title'],
        results['items'][0]['statistics']['viewCount']))


In [41]:
##TESTS to see if we can access YouTube Data API Calls
youtube = get_authenticated_service()

if __name__ == '__main__':
  # When running locally, disable OAuthlib's HTTPs verification. When
  # running in production *do not* leave this option enabled.
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
  channels_list_by_username(youtube,
      part='snippet,contentDetails,statistics',
      forUsername='GoogleDevelopers')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=962228416679-a2a9bv9bc318gjogfv2e89advae97c46.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=ceGLYcpjVhdWHUISCcQR3tJxRq8pZa&prompt=consent&access_type=offline
Enter the authorization code: 4/NwERq3hACtgYS1aUP6C4kjSPAXR1r0URhAXCQ64GEZYTy_PGA6falOw
This channel's ID is UC_x5XG1OV2P6uZZ5FSM9Ttw. Its title is Google Developers, and it has 154789942 views.


In [42]:
#!/usr/bin/python

# Usage example:
# python comments.py --videoid='<video_id>' --text='<text>'

import httplib2
import os
import sys
import json

from apiclient.discovery import build_from_document
from apiclient.errors import HttpError
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import argparser, run_flow


# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains

# the OAuth 2.0 information for this application, including its client_id and
# client_secret. You can acquire an OAuth 2.0 client ID and client secret from
# the {{ Google Cloud Console }} at
# {{ https://cloud.google.com/console }}.
# Please ensure that you have enabled the YouTube Data API for your project.
# For more information about using OAuth2 to access the YouTube Data API, see:
#   https://developers.google.com/youtube/v3/guides/authentication
# For more information about the client_secrets.json file format, see:
#   https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
YOUTUBE_READ_WRITE_SSL_SCOPE = "https://www.googleapis.com/auth/youtube.force-ssl"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [43]:
# Call the API's commentThreads.list method to list the existing comment threads.
def get_comment_threads(youtube, video_id):
  results = youtube.commentThreads().list(
    part="snippet",
    videoId=video_id,
    textFormat="plainText"
  ).execute()

  for item in results["items"]:
    comment = item["snippet"]["topLevelComment"]
    author = comment["snippet"]["authorDisplayName"]
    text = comment["snippet"]["textDisplay"]
    print ("Comment by %s: %s" % (author, text))

  return results["items"]


# Call the API's comments.list method to list the existing comment replies.
def get_comments(youtube, parent_id):
  results = youtube.comments().list(
    #Change to 'full sample?'
    part="snippet",
    parentId=parent_id,
    textFormat="plainText"
  ).execute()

  for item in results["items"]:
    author = item["snippet"]["authorDisplayName"]
    text = item["snippet"]["textDisplay"]
    print ("Comment by %s: %s" % (author, text))

  return results["items"]


In [7]:
##This returns the Comments for a single Youtube Video

if __name__ == "__main__":
  #Change Video ID : "PDqGZ89-O4I"
  video_comment_threads = get_comment_threads(youtube, "PDqGZ89-O4I")
  parent_id = video_comment_threads[0]["id"]
  video_comments = get_comments(youtube, parent_id)
    

Comment by red cat: wow rare video with different screen language and people commenting with English.
Comment by Haerul Muttaqin: thanks Google 😘
Comment by INTOR Info dan Tutorial: Ok google
Comment by Boby Kurniawan Nugraha: I wish i can learn at google too
Comment by Matiullah Karimi: We, Afghans hope to have such a program in Afghanistan too.
Comment by Glamvian Hermawan: Tnankyou google
Comment by Auzi Asfarian: Glad to be a part of this program!
Comment by download free apps and games mod: Developer
Comment by Kavin Raju S: That's Great to enthusiastic developers.
Comment by Aovin Mahmud: Love for indonesian developers from our *BANGLADESH*
Comment by muzzamil ahmed: What needs is real passion , she can't hear but she can achieve❤
Comment by Hari Anugrah: and here I am wondering
Why didn't I be there?
Comment by D'PONSEL: Good job 🇮🇩
Comment by DIRK BIROT: Can you recommend a good smart phone android ide
Comment by Giselle Alessandra Velazquez: This is great! Regardless the fact 

In [44]:
# Call the API's commentThreads.list method to list the existing comments.
def get_comments(youtube, video_id, channel_id):
  #Name of the file to Open and begin writing
      results = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        channelId=channel_id,
        maxResults = 100,
        textFormat="plainText"
      ).execute()
      #Have to use this format, since json.load() function only allows the load of one json object, 
      #since there are many that are created during this process, we essentially have 
      #to trick the method with the [', ' ... ] format, giving the impression that there is only one.  
        
      #Would give a json decoding error, TOO MUCH DATA
      return results

# Call the API's commentThreads.list method to list the existing comments.
def get_next_comments(youtube, video_id, channel_id, nextPage):
  #Name of the file to Open and begin writing
      results = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        channelId=channel_id,
        maxResults = 100,
        pageToken= nextPage,
        textFormat="plainText"
      ).execute()
      #Have to use this format, since json.load() function only allows the load of one json object, 
      #since there are many that are created during this process, we essentially have 
      #to trick the method with the [', ' ... ] format, giving the impression that there is only one.  
        
      #Would give a json decoding error, TOO MUCH DATA
      return results


In [45]:
#Loads comments from the json file
import json
def load_comments(match, f, point):
    for item in match["items"]:
        comment = item["snippet"]["topLevelComment"]
        author = comment["snippet"]["authorDisplayName"]
        text = comment["snippet"]["textDisplay"]
        if (point != 0):
            f.write(', ' + json.dumps(item))
        else:
            f.write(json.dumps(item))
            point = point + 1

In [70]:
import json
##Prints the comments from all 16 Channels into separate json log files
if __name__ == "__main__":
    
  #BATTLE ROYALE
  #Fortnite = '2gUtfBmw86Y' ############
  #ApexLegends = 'G9xpYNGRxbE'  ############
  #Blackout = 'TktjCqG4p7Y' ###########
  #Pupg = '4rG9noTfb4A' ##############

  #SPORTS
  #Rocket League = 'weV-WToHdhY' ###############
  #FIFA 15 = 'TnTYVT6lwBc' ##############
  #Madden 19 = '0VbE2yafP5I' #############
  #2k16 = 'ZKcHuHmfMeE' ###########
    
  #FPS
  #titanfall = 'z5sqPxbLmIU' ############
  #COD = '6kqe2ICmTxc' ##############
  #BATTLEFIELD = 'fb1MR85XFOc' ############
  #Overwatch = 'FqnKB22pOC0' #############

  #SINGLE PLAYER
  #Skyrim = 'PjqsYzBrP-M' ###########
  #Fallout = 'M9FGaan35s0' ############
  #GTA 5 = 'VjZ5tgjPVfU' ###########
  #WitchHunter = 'XHrskkHf958' #########
    

  channels = ['-pUhraVG7Ow']
  json_logs = ['assassins4Comments.json']
  count = 0
  fileNumber = 0
  for channel in channels: 
     point = 0
     fname = json_logs[fileNumber]
     with open(fname, 'w', encoding="utf-8") as f:
        f.write('[')
        channel_comments = get_comments(youtube,channel, None) 
        next_page_token = channel_comments["nextPageToken"]
        #For first call, don't print ','
        load_comments(channel_comments, f, point)
        point = point + 1
        while next_page_token and count < 249:
            try:
                next_page_token = channel_comments["nextPageToken"]
                channel_comments = get_next_comments(youtube, channel, None, next_page_token)
                load_comments(channel_comments, f, point)
                count = count + 1
            except KeyError as e:
                print ("Done")
                break;
        f.write(']')
     fileNumber = fileNumber + 1
print(count)
    
    

249


In [ ]:
#Reads comments from all json files 
def read_comments_from_json(filename):
    comments=[]
    commentCount = 0
    with io.open(filename) as f:
        data=json.load(f)
        for comment in data:
            #Adds only the textDisplay for each comment        
            comments.append(comment["snippet"]["topLevelComment"]["snippet"]["textDisplay"])
            commentCount = commentCount + 1
    print (commentCount)
    return comments  

In [11]:
import nltk, re, json, io #how to download NLTK, see https://www.nltk.org/data.html 

#Creates list of all comments
fortnite_comments = read_comments_from_json('fortniteComments.json')
#apex_Comments = read_comments_from_json('apexComments.json')
#cod_comments = read_comments_from_json('CODComments.json')
#battlefield_comments = read_comments_from_json('battleFieldComments.json')
#fifa_comments = read_comments_from_json('fifaComments.json')
#madden_comments = read_comments_from_json('maddenComments.json')

NameError: name 'read_comments_from_json' is not defined

In [16]:
print(fortnite_comments)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
print(apexLegends_comments)

NameError: name 'apexLegends_comments' is not defined

In [ ]:
print(rocketLeague_comments)

In [ ]:
print(leagueOfLegends_comments)